In [ ]:
# testing the model

winter_model = tf.keras.models.load_model('winter.keras')


x_test_winter_transformed = scaler_x_winter.transform(x_test_winter)
y_test_winter_transformed = scaler_y_winter.transform(y_test_winter)

# Criar janelas deslizantes

timesteps = 300  # Número de passos no tempo
n_features = 5  # Número de features
y_timesteps = 60


# Criar janelas deslizantes
X_aux, y_aux = [], []
for i in range(len(x_test_winter_transformed) - timesteps - y_timesteps):
    X_aux.append(x_test_winter_transformed[i : i + timesteps, :])  # 300 timesteps com 5 features
    y_aux.append(
        y_test_winter_transformed[i + timesteps + 1 : i + timesteps + y_timesteps + 1]
    )  # Potência como alvo (ajuste conforme necessário)

X_aux = np.array(X_aux)
y_aux = np.array(y_aux)

y_pred = winter_model.predict(X_aux)

y_pred = scaler_y_winter.inverse_transform(y_pred)

# y_aux = scaler_y_winter.inverse_transform(y_aux)


In [ ]:
import plotly.graph_objects as go
import numpy as np

# Example index to visualize
sample_index = 5500

# Extract the sample from training data
X_sample = X_aux[sample_index]  # Shape: (300, 5)
y_sample = y_aux[sample_index]  # Shape: (60,)
y_sample_pred = y_pred[sample_index].reshape(1,-1)  # Shape: (60,)

# Inverse transform the sample
X_sample = scaler_x_winter.inverse_transform(X_sample)
y_sample = scaler_y_winter.inverse_transform(y_sample)
# y_sample_pred = scaler_y_winter.inverse_transform(y_sample_pred)

# Create a figure
fig = go.Figure()

# Plot each feature in X_sample
for feature in range(X_sample.shape[1]):
    fig.add_trace(
        go.Scatter(
            x=list(range(X_sample.shape[0])),
            y=X_sample[:, feature],
            mode="lines",
            name=f"Feature {feature+1}",
        )
    )

# Plot the target (y_sample)
fig.add_trace(
    go.Scatter(
        x=list(range(X_sample.shape[0], X_sample.shape[0] + y_sample.shape[0])),
        y=y_sample.flatten(),
        mode="lines",
        name="Target (y)",
        line=dict(dash="dash", width=2, color="black"),
    )
)

fig.add_trace(
    go.Scatter(
        x=list(range(X_sample.shape[0], X_sample.shape[0] + y_sample.shape[0])),
        y=y_sample_pred.flatten(),
        mode="lines",
        name="Target predicted",
        line=dict(dash="dash", width=2, color="yellow"),
    )
)

# Customize the plot
fig.update_layout(
    title=f"Visualization of Sample {sample_index}",
    xaxis_title="Timesteps",
    yaxis_title="Values",
    legend_title="Legend",
    template="plotly_white",
)



fig.show()